In [1]:
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException,NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

import undetected_chromedriver as uc
from time import sleep

In [2]:
tk_fb = os.getenv('TK_FACEBOOK')
mk_fb = os.getenv('MK_FACEBOOK')

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--start-maximized')
driver = uc.Chrome(options=options)
driver.maximize_window()
wait=WebDriverWait(driver, 5)

# Login

In [4]:
driver.get('https://www.facebook.com/')
wait.until(EC.presence_of_element_located((By.XPATH, '//input[@name="email"]'))).send_keys(tk_fb)
wait.until(EC.presence_of_element_located((By.ID, 'pass'))).send_keys(mk_fb)
wait.until(EC.presence_of_element_located((By.NAME, 'login'))).click()

#Crawl newest post_link

In [12]:
driver.get('https://mbasic.facebook.com/jack.phuongtuan1204/')
# driver.get('https://mbasic.facebook.com/MTP.Fan')
wait.until(EC.visibility_of_element_located((By.XPATH,"//*[@id='root']/div/div/div[4]/a"))).click()

In [13]:
all_link_post = []
all_likes_post = []
all_num_cmts_post = []
for i in range(5):
    all_link_ele = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="structured_composer_async_container"]/section/article/footer/div/a[3]')))
    for link in all_link_ele:
        all_link_post.append(link.get_attribute('href'))
        
    all_likes_ele = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='structured_composer_async_container']/section/article/footer/div[2]/span/a[contains(@aria-label, 'Thích')]")))
    for like in all_likes_ele:
        like = like.text.replace(".","")
        all_likes_post.append(int(like))

    all_num_cmts_ele = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//*[@id="structured_composer_async_container"]/section/article/footer/div[2]/a[1]')))
    for num_cmt in all_num_cmts_ele:
        num_cmt = num_cmt.text.replace(".","")
        num_cmt = num_cmt.split(" ")[0]
        all_num_cmts_post.append(int(num_cmt))

    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="structured_composer_async_container"]/div/a'))).click()

display(len(all_link_post))
all_link_post, all_likes_post, all_num_cmts_post

45

(['https://mbasic.facebook.com/story.php?story_fbid=pfbid02vavqEkWkJKC9jB8nht9JKw3n2S2YTuU1k9pQauRUGifg7LPK5t8C81tS6kd8HLfyl&id=100050610514084&eav=Afbnsbj2T8UYCGDk46Gk-m56Jz-uLMRx744kbOHPWGJ02OTBvZFmji93QZ1WFE8MwMY&refid=17&_ft_=encrypted_tracking_data.0AY8aaQsqOwIgKbhEE6HW6m-_W48q8KWcHpBtvPZ9MKJFfqeQGEx8359C0m0yPZYu8r7o1JbhX2zFGKoN0S0tllIE5-bA6OIQJ-7xocZBdPPaR5pRgmvuDQVVv15jO2ysgFha5RvaM9e9JhxTA_sg7sEt5VO9PJF_ZxzDvgEUGDABw4lTDFZ0EMtwETzSyVf41w46epjrMxpKkV-zrTNaUdv08lETEXWVACIapZ033Sq6msU44VKG3Pij5bxW79TkDEQ5irsst_WqPcwKrDaCtBjApX3KSTlS7nHWmxCzmjemYK9RnMQ8vLza76Yoq7SOHkTOMAyoC3qbeOvwnU74QgD024cXY2vd2tG2zRDoqKGh9pSBHrfG2aCjuBssSa-tXATocOU7ejgpwbxksX7q16Gg-FS9o5Fb7Oe8udHOTLKYlyRWHVCT7L3LUtoR5UUeVFDHgYenUOz_68ZgJdVxnp7jSrt8LbNxn5s-4Wg1J3ohLq8b4G2JmtKwgtuu686aa-2tdxjTVGE7VeHO0ICRZi-0TesdDrtzuvJmSSAztaLhRoI7Hgd0Q8vDElQgqE6w2hDcFxedihDNNMW18chtgerZLLYjg67uykG4xxaA3GLDI26X6xiUWIV8_o1Bs8I_9nVxtgMba9_oOqdpyX4MdcbO2RZUS3ikKR8buNk_1hkootyTqvhQ9Lhs8c9LXs49aUUWmkys8DbmW_vYG1dCG7XZJ1BJf_mPWpsLJBu4J

# Crawl_cmt in each post

In [14]:
data_post = {}
for k,l,c in zip(all_link_post, all_likes_post, all_num_cmts_post):
    driver.get(k)
    # try:
    #     title_post = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='m_story_permalink_view']/div/div/div/div/div/p"))).text
    # except TimeoutException:
    #     title_post = "@@This post dont have title@@"
    if 'photo' in k:
        all_cmt_ele = wait.until(EC.visibility_of_any_elements_located((By.XPATH, "//*[@id='MPhotoContent']/div[2]/div/div/div[4]/div/div")))
        all_cmt = [cmt.text for cmt in all_cmt_ele]
    else:
        all_cmt_ele = wait.until(EC.visibility_of_any_elements_located((By.XPATH, "//*[@id='m_story_permalink_view']/div[2]/div/div[5]/div/*/div[1]")))
        all_cmt = [cmt.text for cmt in all_cmt_ele]
    # display(all_cmt)
    data_post[k] = {"likes":l, "num_cmts":c, "cmts":all_cmt}
data_post

{'https://mbasic.facebook.com/story.php?story_fbid=pfbid02vavqEkWkJKC9jB8nht9JKw3n2S2YTuU1k9pQauRUGifg7LPK5t8C81tS6kd8HLfyl&id=100050610514084&eav=Afbnsbj2T8UYCGDk46Gk-m56Jz-uLMRx744kbOHPWGJ02OTBvZFmji93QZ1WFE8MwMY&refid=17&_ft_=encrypted_tracking_data.0AY8aaQsqOwIgKbhEE6HW6m-_W48q8KWcHpBtvPZ9MKJFfqeQGEx8359C0m0yPZYu8r7o1JbhX2zFGKoN0S0tllIE5-bA6OIQJ-7xocZBdPPaR5pRgmvuDQVVv15jO2ysgFha5RvaM9e9JhxTA_sg7sEt5VO9PJF_ZxzDvgEUGDABw4lTDFZ0EMtwETzSyVf41w46epjrMxpKkV-zrTNaUdv08lETEXWVACIapZ033Sq6msU44VKG3Pij5bxW79TkDEQ5irsst_WqPcwKrDaCtBjApX3KSTlS7nHWmxCzmjemYK9RnMQ8vLza76Yoq7SOHkTOMAyoC3qbeOvwnU74QgD024cXY2vd2tG2zRDoqKGh9pSBHrfG2aCjuBssSa-tXATocOU7ejgpwbxksX7q16Gg-FS9o5Fb7Oe8udHOTLKYlyRWHVCT7L3LUtoR5UUeVFDHgYenUOz_68ZgJdVxnp7jSrt8LbNxn5s-4Wg1J3ohLq8b4G2JmtKwgtuu686aa-2tdxjTVGE7VeHO0ICRZi-0TesdDrtzuvJmSSAztaLhRoI7Hgd0Q8vDElQgqE6w2hDcFxedihDNNMW18chtgerZLLYjg67uykG4xxaA3GLDI26X6xiUWIV8_o1Bs8I_9nVxtgMba9_oOqdpyX4MdcbO2RZUS3ikKR8buNk_1hkootyTqvhQ9Lhs8c9LXs49aUUWmkys8DbmW_vYG1dCG7XZJ1BJf_mPWpsLJBu4Jw

In [20]:
data_post

{'https://mbasic.facebook.com/story.php?story_fbid=pfbid02vavqEkWkJKC9jB8nht9JKw3n2S2YTuU1k9pQauRUGifg7LPK5t8C81tS6kd8HLfyl&id=100050610514084&eav=Afbnsbj2T8UYCGDk46Gk-m56Jz-uLMRx744kbOHPWGJ02OTBvZFmji93QZ1WFE8MwMY&refid=17&_ft_=encrypted_tracking_data.0AY8aaQsqOwIgKbhEE6HW6m-_W48q8KWcHpBtvPZ9MKJFfqeQGEx8359C0m0yPZYu8r7o1JbhX2zFGKoN0S0tllIE5-bA6OIQJ-7xocZBdPPaR5pRgmvuDQVVv15jO2ysgFha5RvaM9e9JhxTA_sg7sEt5VO9PJF_ZxzDvgEUGDABw4lTDFZ0EMtwETzSyVf41w46epjrMxpKkV-zrTNaUdv08lETEXWVACIapZ033Sq6msU44VKG3Pij5bxW79TkDEQ5irsst_WqPcwKrDaCtBjApX3KSTlS7nHWmxCzmjemYK9RnMQ8vLza76Yoq7SOHkTOMAyoC3qbeOvwnU74QgD024cXY2vd2tG2zRDoqKGh9pSBHrfG2aCjuBssSa-tXATocOU7ejgpwbxksX7q16Gg-FS9o5Fb7Oe8udHOTLKYlyRWHVCT7L3LUtoR5UUeVFDHgYenUOz_68ZgJdVxnp7jSrt8LbNxn5s-4Wg1J3ohLq8b4G2JmtKwgtuu686aa-2tdxjTVGE7VeHO0ICRZi-0TesdDrtzuvJmSSAztaLhRoI7Hgd0Q8vDElQgqE6w2hDcFxedihDNNMW18chtgerZLLYjg67uykG4xxaA3GLDI26X6xiUWIV8_o1Bs8I_9nVxtgMba9_oOqdpyX4MdcbO2RZUS3ikKR8buNk_1hkootyTqvhQ9Lhs8c9LXs49aUUWmkys8DbmW_vYG1dCG7XZJ1BJf_mPWpsLJBu4Jw

In [22]:
a=data_post
a

{'https://mbasic.facebook.com/story.php?story_fbid=pfbid02vavqEkWkJKC9jB8nht9JKw3n2S2YTuU1k9pQauRUGifg7LPK5t8C81tS6kd8HLfyl&id=100050610514084&eav=Afbnsbj2T8UYCGDk46Gk-m56Jz-uLMRx744kbOHPWGJ02OTBvZFmji93QZ1WFE8MwMY&refid=17&_ft_=encrypted_tracking_data.0AY8aaQsqOwIgKbhEE6HW6m-_W48q8KWcHpBtvPZ9MKJFfqeQGEx8359C0m0yPZYu8r7o1JbhX2zFGKoN0S0tllIE5-bA6OIQJ-7xocZBdPPaR5pRgmvuDQVVv15jO2ysgFha5RvaM9e9JhxTA_sg7sEt5VO9PJF_ZxzDvgEUGDABw4lTDFZ0EMtwETzSyVf41w46epjrMxpKkV-zrTNaUdv08lETEXWVACIapZ033Sq6msU44VKG3Pij5bxW79TkDEQ5irsst_WqPcwKrDaCtBjApX3KSTlS7nHWmxCzmjemYK9RnMQ8vLza76Yoq7SOHkTOMAyoC3qbeOvwnU74QgD024cXY2vd2tG2zRDoqKGh9pSBHrfG2aCjuBssSa-tXATocOU7ejgpwbxksX7q16Gg-FS9o5Fb7Oe8udHOTLKYlyRWHVCT7L3LUtoR5UUeVFDHgYenUOz_68ZgJdVxnp7jSrt8LbNxn5s-4Wg1J3ohLq8b4G2JmtKwgtuu686aa-2tdxjTVGE7VeHO0ICRZi-0TesdDrtzuvJmSSAztaLhRoI7Hgd0Q8vDElQgqE6w2hDcFxedihDNNMW18chtgerZLLYjg67uykG4xxaA3GLDI26X6xiUWIV8_o1Bs8I_9nVxtgMba9_oOqdpyX4MdcbO2RZUS3ikKR8buNk_1hkootyTqvhQ9Lhs8c9LXs49aUUWmkys8DbmW_vYG1dCG7XZJ1BJf_mPWpsLJBu4Jw

In [36]:
import json
with open('data_post.json', 'w',encoding='utf-8') as f:
    json.dump(a, f, indent=4)

In [40]:
with open('data_post.json', 'r',encoding='utf-8') as f:
    data = json.load(f)
data.values()

dict_values([{'likes': 23894, 'num_cmts': 1839, 'cmts': ['Nho chong iu qua ah', 'Màu đen sang trọng. Quý phái. Đẹp đẹp đẹp', 'Zaiii đẹp nhà mìnhhhh 🥰', 'Chán thằng em có tuổi mà không khôn tí nào', 'Lại sắp có mv mới 😏', 'Dễ thương', 'Jack đẹp trai và cầu thủ Veratti của PSG 🥰', 'Tuyệt vời', 'Lại sắp ghép vào MV:))', 'Mãi iu a Jack \U0001faf6❤️']}, {'likes': 23167, 'num_cmts': 1764, 'cmts': ['Xinh yêu của Đóm đây rồiiii hì hì 😋', 'Mãi yêu thương tứn🐧', 'Bài đặc cảm ơn 😠', 'Bài đặc cảm ơn 😠', 'Thật hạnh phúc nha hong', 'Hihi', 'nhìn xem, anh ta đang cười vì hạnh phúc kìaaaaa', 'Đẹp traii', 'Thương abe rất nhiều 🌻🥰', 'rất nhiều đóm vào đây cmt']}, {'likes': 19035, 'num_cmts': 1285, 'cmts': ['Luôn yêu ❤', 'Luôn hướng về anh, luôn ủng hộ, yêu thương anh❤️😘😘', 'Luôn yêu 🐧🐧👽👽', 'Luôn yêu thương 🥰🥰', 'Luôn yêu thương Meoo💗❤', 'Luôn yêu thương Meoo 💗❤', 'luôn yêu 🐸', 'Luôn yêu thương meomeo💗', 'Luôn yêu 😋', 'Luôn yêu meo']}, {'likes': 54824, 'num_cmts': 3496, 'cmts': ['Luôn yêu thương a 🪰✨', '

In [43]:
cmts_values = []
for _, dict_val in data.items():
    cmts_values.append(dict_val['cmts'])
cmts_values

[['Nho chong iu qua ah',
  'Màu đen sang trọng. Quý phái. Đẹp đẹp đẹp',
  'Zaiii đẹp nhà mìnhhhh 🥰',
  'Chán thằng em có tuổi mà không khôn tí nào',
  'Lại sắp có mv mới 😏',
  'Dễ thương',
  'Jack đẹp trai và cầu thủ Veratti của PSG 🥰',
  'Tuyệt vời',
  'Lại sắp ghép vào MV:))',
  'Mãi iu a Jack \U0001faf6❤️'],
 ['Xinh yêu của Đóm đây rồiiii hì hì 😋',
  'Mãi yêu thương tứn🐧',
  'Bài đặc cảm ơn 😠',
  'Bài đặc cảm ơn 😠',
  'Thật hạnh phúc nha hong',
  'Hihi',
  'nhìn xem, anh ta đang cười vì hạnh phúc kìaaaaa',
  'Đẹp traii',
  'Thương abe rất nhiều 🌻🥰',
  'rất nhiều đóm vào đây cmt'],
 ['Luôn yêu ❤',
  'Luôn hướng về anh, luôn ủng hộ, yêu thương anh❤️😘😘',
  'Luôn yêu 🐧🐧👽👽',
  'Luôn yêu thương 🥰🥰',
  'Luôn yêu thương Meoo💗❤',
  'Luôn yêu thương Meoo 💗❤',
  'luôn yêu 🐸',
  'Luôn yêu thương meomeo💗',
  'Luôn yêu 😋',
  'Luôn yêu meo'],
 ['Luôn yêu thương a 🪰✨',
  'Sinh nhật ngta mà nhiều đứa vô thả phẩn nộ kì cục thật',
  'Cái gì zay',
  'Luôn yêu thương anh Jack \U0001faf6🏻',
  '1 đêm sinh

In [83]:
try:
    title_post = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='m_story_permalink_view']/div/div/div/div/div/p"))).text
except TimeoutException:
    title_post = "@@This post dont have title@@"
title_post

'@@This post dont have title@@'

In [68]:
all_cmt_ele = wait.until(EC.visibility_of_any_elements_located((By.XPATH, "//*[@id='m_story_permalink_view']/div[2]/div/div[5]/div/*/div[1]")))
all_cmt = [cmt.text for cmt in all_cmt_ele]
all_cmt

['xíu nữa thoiii:))',
 'xíu nữa hoiii:) ❤️',
 'Một Xíu nữa thoiii 🌻❤',
 'Xíu nữa thoiiii:)))',
 'Xíu nữa thoiii',
 'Ngóng lắm luon ý',
 'Xíu xíu nữa',
 'yêu thương',
 'Hóng quó ii thoiii',
 'hóng quá chòi oiii']